### System settings verification

In [1]:
import sys
sys.version

'3.6.5 | packaged by conda-forge | (default, Apr  6 2018, 13:44:09) \n[GCC 4.2.1 Compatible Apple LLVM 6.1.0 (clang-602.0.53)]'

In [2]:
!which python
!echo $PYTHONPATH
!java --version

/Users/harjeet/DevEnv/miniconda3/bin/python

java 10.0.1 2018-04-17
Java(TM) SE Runtime Environment 18.3 (build 10.0.1+10)
Java HotSpot(TM) 64-Bit Server VM 18.3 (build 10.0.1+10, mixed mode)


# Creating a Spark Context

In [3]:
from pyspark.sql import SparkSession, DataFrameReader, Row
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [4]:
spark = SparkSession\
    .builder\
    .config("master","local[*]")\
    .appName("LastFM")\
    .getOrCreate()

In [5]:
spark.sparkContext._conf.getAll()

[('spark.driver.host', 'harjeets-macbook-pro.local'),
 ('spark.app.id', 'local-1527099709450'),
 ('master', 'local[*]'),
 ('spark.driver.port', '61865'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.app.name', 'LastFM')]

# Getting the data
Here's the [README](http://files.grouplens.org/datasets/hetrec2011/hetrec2011-lastfm-readme.txt) for a description.
Briefly:
- 92834 "scrobbles" (play counts) per user of an artist
- 1892 users
- 17632 artists
- density/sparsity of the data: 0.28%

In [7]:
import wget
url = 'http://files.grouplens.org/datasets/hetrec2011/hetrec2011-lastfm-2k.zip'
filename = wget.download(url)
filename

'hetrec2011-lastfm-2k.zip'

In [8]:
import zipfile
with zipfile.ZipFile(filename,"r") as zip_ref:
    zip_ref.extractall("./data")
!ls ./data

artists.dat                       user_friends.dat
readme.txt                        user_taggedartists-timestamps.dat
tags.dat                          user_taggedartists.dat
user_artists.dat


In [6]:
import glob
# files_to_process = [os.path.relpath(dat_path, './data') for dat_path in glob.glob('./data/*.dat')]
files_to_process = glob.glob('./data/*.dat')
files_to_process

['./data/user_friends.dat',
 './data/user_taggedartists-timestamps.dat',
 './data/artists.dat',
 './data/tags.dat',
 './data/user_taggedartists.dat',
 './data/user_artists.dat']

In [7]:
# Work in progress
def read_dats(filename):
#     df  = spark.read.format("text").load(filename)
    df  = spark.read.option("delimiter", "\t").csv(filename, header=True)
    return df

In [8]:
read_dats('./data/artists.dat').show(10)

+---+-----------------+--------------------+--------------------+
| id|             name|                 url|          pictureURL|
+---+-----------------+--------------------+--------------------+
|  1|     MALICE MIZER|http://www.last.f...|http://userserve-...|
|  2|  Diary of Dreams|http://www.last.f...|http://userserve-...|
|  3|Carpathian Forest|http://www.last.f...|http://userserve-...|
|  4|     Moi dix Mois|http://www.last.f...|http://userserve-...|
|  5|      Bella Morte|http://www.last.f...|http://userserve-...|
|  6|        Moonspell|http://www.last.f...|http://userserve-...|
|  7|   Marilyn Manson|http://www.last.f...|http://userserve-...|
|  8|      DIR EN GREY|http://www.last.f...|http://userserve-...|
|  9|      Combichrist|http://www.last.f...|http://userserve-...|
| 10|          Grendel|http://www.last.f...|http://userserve-...|
+---+-----------------+--------------------+--------------------+
only showing top 10 rows



In [11]:
dist_files_to_process = sc.parallelize(files_to_process)
processed = dist_files_to_process.map(lambda file: read_dats(file))
collected = processed.collect()

NameError: name 'sc' is not defined

In [ ]:
def my_func(file_to_process):
    # do something with file
    return x, y

conf = SparkConf()
conf.setMaster(mySparkMasterUrl)
conf.setAppName('Simple app')
sc = SparkContext(conf=conf)

files_to_process = ['s:/file1.DAT', 's:/file2.DAT', 's:/file3.DAT', 's:/file4.DAT']
dist_files_to_process = sc.parallelize(files_to_process)
processed = dist_files_to_process.map(lambda file: my_func(file))
collected = processed.collect()